# Ecommerce Analytics Dashboard
Interactive visualizations for KPIs, revenue trends, top products, and conversion funnel

In [1]:
# Install plotly if not already installed
try:
    import plotly
except ImportError:
    import sys
    !{sys.executable} -m pip install plotly
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import numpy as np

# Set up paths
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "Data"
PROCESSED_DIR = DATA_DIR / "Processed"
ANALYTICS_DIR = DATA_DIR / "analytics"

print(f"Project Root: {PROJECT_ROOT}")
print(f"Data Directory: {DATA_DIR}")

Project Root: c:\Python\ecommerce_etl_project
Data Directory: c:\Python\ecommerce_etl_project\Data


In [2]:
# Load processed data
dim_users = pd.read_csv(PROCESSED_DIR / "dim_users.csv")
dim_products = pd.read_csv(PROCESSED_DIR / "dim_products.csv")
fact_orders = pd.read_csv(PROCESSED_DIR / "fact_orders.csv")
fact_order_items = pd.read_csv(PROCESSED_DIR / "fact_order_items.csv")
fact_events = pd.read_csv(PROCESSED_DIR / "fact_events.csv")
fact_reviews = pd.read_csv(PROCESSED_DIR / "fact_reviews.csv")

print("Data loaded successfully!")
print(f"\nDatasets:")
print(f"  - dim_users: {dim_users.shape[0]} rows")
print(f"  - dim_products: {dim_products.shape[0]} rows")
print(f"  - fact_orders: {fact_orders.shape[0]} rows")
print(f"  - fact_order_items: {fact_order_items.shape[0]} rows")
print(f"  - fact_events: {fact_events.shape[0]} rows")

Data loaded successfully!

Datasets:
  - dim_users: 10000 rows
  - dim_products: 2000 rows
  - fact_orders: 20000 rows
  - fact_order_items: 43525 rows
  - fact_events: 80000 rows


## Key Performance Indicators (KPIs)

In [3]:
# Calculate KPIs
completed = fact_orders[fact_orders["order_status"].astype(str).str.lower() == "completed"].copy()
completed["total_amount"] = pd.to_numeric(completed["total_amount"], errors="coerce").fillna(0)

total_revenue = completed["total_amount"].sum()
total_orders = completed.shape[0]
avg_order_value = total_revenue / total_orders if total_orders else 0
total_customers = dim_users["user_id"].nunique()
orders_per_customer = total_orders / total_customers if total_customers else 0

# Create KPI display
kpi_data = {
    "Total Revenue": f"${total_revenue:,.2f}",
    "Total Orders": f"{total_orders:,}",
    "Avg Order Value": f"${avg_order_value:.2f}",
    "Total Customers": f"{total_customers:,}",
    "Orders per Customer": f"{orders_per_customer:.2f}"
}

print("\n" + "="*50)
print("KEY PERFORMANCE INDICATORS")
print("="*50)
for key, value in kpi_data.items():
    print(f"{key:.<30} {value:>15}")
print("="*50)


KEY PERFORMANCE INDICATORS
Total Revenue.................   $2,419,712.58
Total Orders..................           4,021
Avg Order Value...............         $601.77
Total Customers...............          10,000
Orders per Customer...........            0.40


In [4]:
# Create KPI cards visualization
def safe_float(val):
    try:
        return float(val)
    except Exception:
        return 0.0

kpi_metrics = [
    {"label": "Total Revenue", "value": safe_float(total_revenue), "color": "#1f77b4"},
    {"label": "Total Orders", "value": safe_float(total_orders), "color": "#ff7f0e"},
    {"label": "Avg Order Value", "value": safe_float(avg_order_value), "color": "#2ca02c"},
    {"label": "Total Customers", "value": safe_float(total_customers), "color": "#d62728"},
]

fig = go.Figure()

for i, metric in enumerate(kpi_metrics):
    fig.add_trace(go.Indicator(
        mode="number",
        value=metric["value"],
        title={"text": metric["label"]},
        domain={"x": [i * 0.25, (i + 1) * 0.25], "y": [0, 1]}
    ))

fig.update_layout(
    title="KPIs Overview",
    height=300,
    showlegend=False,
    grid={"rows": 1, "columns": 4}
    # No need for color here, but you can style further if needed
    # color=metric["color"]
    # This is not a valid argument for update_layout
    # If you want to color the numbers, use font or indicator styling
    # See Plotly docs for more advanced styling
    # https://plotly.com/python-api-reference/generated/plotly.graph_objects.Indicator.html
    # https://plotly.com/python-api-reference/generated/plotly.graph_objects.layout.html
    # https://plotly.com/python-api-reference/generated/plotly.graph_objects.layout.Font.html
)
fig.show()

## Top 10 Products by Revenue

In [5]:
# Top products by revenue
foi = fact_order_items.copy()
foi["item_price"] = pd.to_numeric(foi["item_price"], errors="coerce").fillna(0)
foi["quantity"] = pd.to_numeric(foi["quantity"], errors="coerce").fillna(0)
foi["item_total"] = foi["item_price"] * foi["quantity"]

prod_rev = foi.groupby("product_id", as_index=False)["item_total"].sum().rename(columns={"item_total": "revenue"})
prod_rev = prod_rev.merge(dim_products[["product_id", "product_name"]], on="product_id", how="left")
top10 = prod_rev.sort_values("revenue", ascending=False).head(10)

print("\nTop 10 Products by Revenue:")
print(top10.to_string(index=False))


Top 10 Products by Revenue:
product_id  revenue    product_name
   P001527 73275.12   Willow Result
   P001024 65467.64      Astra Pull
   P000021 62032.60  Willow Special
   P000910 60784.10     Orion Group
   P000136 56419.48    Zenith Phone
   P001925 56311.92 Willow Hospital
   P000461 54411.20   Nimbus Family
   P001049 54399.32    Zenith Their
   P000198 54105.20      Orion Face
   P000555 53489.18         Acme Of


In [6]:
# Visualize top 10 products
if not top10.empty and "product_name" in top10.columns and "revenue" in top10.columns:
    fig = px.bar(
        top10,
        x="revenue",
        y="product_name",
        orientation="h",
        title="Top 10 Products by Revenue",
        labels={"revenue": "Revenue ($)", "product_name": "Product"},
        color="revenue",
        color_continuous_scale="Blues"
    )
    fig.update_layout(height=500, showlegend=False)
    fig.update_xaxes(title="Revenue ($)")
    fig.update_yaxes(title="")
    fig.show()
else:
    print("No data available for Top 10 Products visualization.")

## Monthly Revenue Trend

In [7]:
# Monthly revenue trend
completed["order_date"] = pd.to_datetime(completed["order_date"], errors="coerce")
monthly_rev = completed.groupby(completed["order_date"].dt.to_period("M"))["total_amount"].sum().reset_index()
monthly_rev.columns = ["month", "revenue"]
monthly_rev["month"] = monthly_rev["month"].dt.to_timestamp()

print("\nMonthly Revenue:")
print(monthly_rev.to_string(index=False))


Monthly Revenue:
     month   revenue
2024-01-01  95429.66
2024-02-01  73319.35
2024-03-01 134569.53
2024-04-01 103610.56
2024-05-01 115249.52
2024-06-01 126550.29
2024-07-01 108635.16
2024-08-01  91515.76
2024-09-01  96088.95
2024-10-01 113797.42
2024-11-01  98829.66
2024-12-01 109147.67
2025-01-01  84532.56
2025-02-01 109610.71
2025-03-01  81746.13
2025-04-01 114765.65
2025-05-01 126909.41
2025-06-01 116163.49
2025-07-01 111926.51
2025-08-01 109437.47
2025-09-01 115742.79
2025-10-01 111493.71
2025-11-01  70640.62


In [8]:
# Plot monthly revenue trend
if not monthly_rev.empty and "month" in monthly_rev.columns and "revenue" in monthly_rev.columns:
    fig = px.line(
        monthly_rev,
        x="month",
        y="revenue",
        title="Monthly Revenue Trend",
        labels={"month": "Month", "revenue": "Revenue ($)"},
        markers=True,
        line_shape="spline"
    )
    fig.update_layout(height=500, hovermode="x unified")
    fig.update_traces(line=dict(color="#1f77b4", width=3), marker=dict(size=8))
    fig.show()
else:
    print("No data available for Monthly Revenue Trend visualization.")

## Conversion Funnel

In [9]:
# Calculate conversion funnel
ev = fact_events.copy()
ev["event_type"] = ev["event_type"].astype(str).str.lower()

views = ev[ev["event_type"] == "view"].shape[0]
carts = ev[ev["event_type"].isin(["cart", "add_to_cart"])].shape[0]
purchases = completed["order_id"].nunique()

funnel_data = pd.DataFrame({
    "stage": ["Views", "Add to Cart", "Purchases"],
    "count": [views, carts, purchases]
})

# Calculate conversion rates
funnel_data["conversion_rate"] = (funnel_data["count"] / funnel_data["count"].iloc[0] * 100).round(2)

print("\nConversion Funnel:")
print(funnel_data.to_string(index=False))


Conversion Funnel:
      stage  count  conversion_rate
      Views  56013           100.00
Add to Cart  12035            21.49
  Purchases   4021             7.18


In [10]:
# Visualize funnel
if not funnel_data.empty and "stage" in funnel_data.columns and "count" in funnel_data.columns:
    fig = go.Figure(go.Funnel(
        y=funnel_data["stage"],
        x=funnel_data["count"],
        textposition="inside",
        textinfo="value+percent initial",
        marker=dict(color=["#1f77b4", "#ff7f0e", "#2ca02c"])
    ))
    fig.update_layout(
        title="Conversion Funnel",
        height=500,
        showlegend=False
    )
    fig.show()
else:
    print("No data available for Conversion Funnel visualization.")

## Product Category Analysis

In [11]:
# Revenue by category
if "category" in dim_products.columns:
    category_rev = foi.merge(dim_products[["product_id", "category"]], on="product_id", how="left")
    category_rev = category_rev.groupby("category")["item_total"].sum().reset_index().sort_values("item_total", ascending=False)
    if not category_rev.empty:
        fig = px.pie(
            category_rev,
            values="item_total",
            names="category",
            title="Revenue Distribution by Category"
        )
        fig.update_layout(height=500)
        fig.show()
    else:
        print("No data available for Category Revenue visualization.")
else:
    print("Category column not available in dim_products")

## Summary Statistics

## Load Data to SQLite Database

In [15]:
# Install required packages
import subprocess
import sys

packages = ['sqlalchemy', 'sqlite3']
for package in packages:
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

In [16]:
# Setup SQLite database with SQLAlchemy
from sqlalchemy import create_engine, inspect
from pathlib import Path
import sqlite3

# Create SQLite database
db_path = PROJECT_ROOT / 'ecommerce.db'
engine = create_engine(f'sqlite:///{db_path}')

print(f"SQLite Database created at: {db_path}")
print(f"Engine: {engine}")

SQLite Database created at: c:\Python\ecommerce_etl_project\ecommerce.db
Engine: Engine(sqlite:///c:\Python\ecommerce_etl_project\ecommerce.db)


In [17]:
# Load CSVs and insert into SQLite
tables_to_load = {
    'dim_users': 'dim_users.csv',
    'dim_products': 'dim_products.csv',
    'fact_order_items': 'fact_order_items.csv',
    'fact_events': 'fact_events.csv',
    'fact_reviews': 'fact_reviews.csv'
}

print("\nLoading data into SQLite database...\n")

for table_name, csv_file in tables_to_load.items():
    csv_path = PROCESSED_DIR / csv_file
    if csv_path.exists():
        print(f"Loading {table_name}...", end=' ')
        df = pd.read_csv(csv_path)
        df.to_sql(table_name, engine, if_exists='replace', index=False)
        print(f"✓ Loaded {df.shape[0]} rows into {table_name}")
    else:
        print(f"⚠ Warning: {csv_file} not found at {csv_path}")

print("\n✓ All data successfully loaded into SQLite!\n")


Loading data into SQLite database...

Loading dim_users... ✓ Loaded 10000 rows into dim_users
Loading dim_products... ✓ Loaded 2000 rows into dim_products
Loading fact_order_items... ✓ Loaded 43525 rows into fact_order_items
Loading fact_events... ✓ Loaded 43525 rows into fact_order_items
Loading fact_events... ✓ Loaded 80000 rows into fact_events
Loading fact_reviews... ✓ Loaded 15000 rows into fact_reviews

✓ All data successfully loaded into SQLite!

✓ Loaded 80000 rows into fact_events
Loading fact_reviews... ✓ Loaded 15000 rows into fact_reviews

✓ All data successfully loaded into SQLite!



In [18]:
# Verify tables in SQLite database
inspector = inspect(engine)
tables = inspector.get_table_names()

print("Tables in SQLite database:")
print("="*60)
for table_name in tables:
    columns = inspector.get_columns(table_name)
    print(f"\n{table_name}:")
    print(f"  Columns: {len(columns)}")
    print(f"  Column Names: {[col['name'] for col in columns]}")

print("\n" + "="*60)
print(f"Total tables: {len(tables)}")

Tables in SQLite database:

dim_products:
  Columns: 6
  Column Names: ['product_id', 'product_name', 'category', 'brand', 'price', 'rating']

dim_users:
  Columns: 8
  Column Names: ['user_id', 'name', 'email', 'gender', 'city', 'signup_date', 'signup_year', 'signup_month']

fact_events:
  Columns: 7
  Column Names: ['event_id', 'user_id', 'product_id', 'event_type', 'event_timestamp', 'event_date', 'event_hour']

fact_order_items:
  Columns: 7
  Column Names: ['order_item_id', 'order_id', 'product_id', 'user_id', 'quantity', 'item_price', 'item_total']

fact_reviews:
  Columns: 7
  Column Names: ['review_id', 'order_id', 'product_id', 'user_id', 'rating', 'review_text', 'review_date']

Total tables: 5


In [ ]:
# Query and display sample data from each table
print("\nSample Data from Each Table:")
print("="*60)

for table_name in tables:
    query = f"SELECT * FROM {table_name} LIMIT 3"
    df_sample = pd.read_sql(query, engine)
    print(f"\n{table_name} (showing first 3 rows):")
    print(df_sample.to_string())
    print()


Sample Data from Each Table:

dim_products (showing first 3 rows):
  product_id     product_name   category    brand   price  rating
0    P000001         Astra Be   Clothing    Astra  157.89    4.08
1    P000002  NeoTech Someone  Groceries  NeoTech   21.46    3.87
2    P000003     Acme Discuss     Sports     Acme  265.37    3.46


dim_users (showing first 3 rows):
   user_id          name                      email gender            city signup_date  signup_year  signup_month
0  U000001    Angel Hill   donaldgarcia@example.net  Other  New Roberttown  2025-03-13         2025             3
1  U000002  Jesse Guzman  jennifermiles@example.com   Male   South Bridget  2024-03-05         2024             3
2  U000003  Adam Shaffer      jpeterson@example.org   Male      Curtisfurt  2025-07-07         2025             7


fact_events (showing first 3 rows):
    event_id  user_id product_id event_type             event_timestamp  event_date  event_hour
0  E00000001  U009798    P001393       car

: 

In [12]:
print("\n" + "="*60)
print("ANALYTICS SUMMARY")
print("="*60)
print(f"\nRevenue Metrics:")
print(f"  Total Revenue: ${total_revenue:,.2f}")
print(f"  Average Order Value: ${avg_order_value:.2f}")
print(f"  Monthly Average: ${monthly_rev['revenue'].mean():,.2f}")
print(f"\nOrder & Customer Metrics:")
print(f"  Total Orders: {total_orders:,}")
print(f"  Total Customers: {total_customers:,}")
print(f"  Orders per Customer: {orders_per_customer:.2f}")
print(f"\nConversion Metrics:")
if views > 0:
    print(f"  View to Cart Rate: {(carts/views*100):.2f}%")
else:
    print("  View to Cart Rate: N/A (no views)")
if carts > 0:
    print(f"  Cart to Purchase Rate: {(purchases/carts*100):.2f}%")
else:
    print("  Cart to Purchase Rate: N/A (no carts)")
if views > 0:
    print(f"  Overall Conversion Rate: {(purchases/views*100):.2f}%")
else:
    print("  Overall Conversion Rate: N/A (no views)")
print("="*60)


ANALYTICS SUMMARY

Revenue Metrics:
  Total Revenue: $2,419,712.58
  Average Order Value: $601.77
  Monthly Average: $105,204.89

Order & Customer Metrics:
  Total Orders: 4,021
  Total Customers: 10,000
  Orders per Customer: 0.40

Conversion Metrics:
  View to Cart Rate: 21.49%
  Cart to Purchase Rate: 33.41%
  Overall Conversion Rate: 7.18%
